In [476]:
import datetime
import math

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import yfinance as yfin

yfin.pdr_override()

In [477]:
end = datetime.date.today()
start = datetime.date.today() - datetime.timedelta(365 * 10)

prices = web.DataReader(["GE"], start, end)["Adj Close"]
prices = pd.DataFrame(prices)
prices

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2013-09-17,120.870552
2013-09-18,122.897446
2013-09-19,121.851250
2013-09-20,119.609543
2013-09-23,120.954597
...,...
2023-09-08,111.720001
2023-09-11,114.260002
2023-09-12,115.000000


In [478]:
initialPrice = prices["Adj Close"]["2013-09-17"]  # we can index with datetime.
initialPrice

120.87055206298828

In [479]:
finalPrice = prices["Adj Close"][-1]  # or we can index normally with int. both will work.
finalPrice

115.6500015258789

We will assume we bought 100 shares 10 years ago. To determine the cash return, we use the following formula:

$r_{_f} = 100\ (p_{_f} - p_{_i})$

In [480]:
print(
    f"With an initial investment of ${(100 * initialPrice):.2f}, \
    \nthe cash return of this investmen would be: \
    \n${((finalPrice - initialPrice) * 100):.3f}"
    )

With an initial investment of $12087.06,     
the cash return of this investmen would be:     
$-522.055


## Return on a Basket of Assets (Portfolio Return) ##

In [481]:
# Construct a portfolio consisting of stocks $META and $CMG

end = datetime.date.today()
start = datetime.date.today() - datetime.timedelta(365 * 10)

prices = web.DataReader(["META", "CMG"], start, end)["Adj Close"]
print("type of prices:", type(prices))
print(prices)

[*********************100%%**********************]  2 of 2 completed
type of prices: <class 'pandas.core.frame.DataFrame'>
                    CMG        META
Date                               
2013-09-17   425.410004   45.070000
2013-09-18   426.230011   45.230000
2013-09-19   421.700012   45.980000
2013-09-20   415.079987   47.490002
2013-09-23   415.239990   47.189999
...                 ...         ...
2023-09-08  1945.099976  297.890015
2023-09-11  1948.849976  307.559998
2023-09-12  1933.010010  301.660004
2023-09-13  1933.040039  305.059998
2023-09-14  1936.119995  311.720001

[2516 rows x 2 columns]


In [482]:
# Again we think about buying 100 shares each 10 years ago.

initialMETA = prices.META[0]
initialCMG = prices.CMG[0]

finalMETA = prices.META[-1]
finalCMG = prices.CMG[-1]

weightMETA = initialMETA / (initialMETA + initialCMG)
weightCMG = initialCMG / (initialMETA + initialCMG)

print(f"We have an initial investment in META of ${(initialMETA * 100):.2f} and in CMG ${(initialCMG * 100):.2f}")
print(f"This would make the weight of META {weightMETA:.3f} and the weight of CMG {weightCMG:.3f}")

We have an initial investment in META of $4507.00 and in CMG $42541.00
This would make the weight of META 0.096 and the weight of CMG 0.904


In [483]:
# Calculate final return of portfolio
RMETA = finalMETA - initialMETA
RCMG = finalCMG - initialCMG

RPortfolio = weightMETA * RMETA + weightCMG * RCMG

print(f"Final portfolio return of the portfolio is ${RPortfolio:.2f}")

Final portfolio return of the portfolio is $1391.53


In [484]:
# Calculate final rate of return of portfolio
rMETA = RMETA / initialMETA - 1
rCMG = RCMG / initialCMG - 1

rPortfolio = weightMETA * rMETA + weightCMG * rCMG

print(f"Final rate of return of portfolio is {rPortfolio:.4f} or {(100*rPortfolio):.2f}%")

Final rate of return of portfolio is 2.7778 or 277.78%


## Calculating Portfolio Variance ##

In [485]:
returns = prices.pct_change()
returns = returns.dropna()
returns

,CMG,META
Date,,
2013-09-18,0.001928,0.003550
2013-09-19,-0.010628,0.016582
2013-09-20,-0.015698,0.032840
2013-09-23,0.000385,-0.006317
2013-09-24,0.008477,0.026701
...,...,...
2023-09-08,-0.011053,-0.002612
2023-09-11,0.001928,0.032462
2023-09-12,-0.008128,-0.019183


In [486]:
returns_variance = np.array(returns.var(ddof=1)) * 252
print("type:",type(returns_variance))
print(returns_variance)

type: <class 'numpy.ndarray'>
[0.12241678 0.14224369]


In [487]:
weights = np.array([weightMETA, weightCMG])
weights_squared = weights**2

covariance = 252 * returns.cov()  # annual covariance
covariance_float = covariance["CMG"]["META"]
print(weights)
print(weights_squared)
print(covariance)
print(covariance_float)

[0.09579578 0.90420422]
[0.00917683 0.81758527]
           CMG      META
CMG   0.122417  0.048005
META  0.048005  0.142244
0.048005150579210086


In [488]:
variance = np.dot(weights_squared, returns_variance) + 2 * weightMETA * weightCMG * covariance_float
print(f"Portfolio variance = {variance:.4f} or {(100*variance):.2f}%")

Portfolio variance = 0.1257 or 12.57%


**Second way to calculate portfolio variance is using covariance matrix and the weights vector**

$M = \begin{bmatrix} 0.122417, 0.048005 \\ 0.048005, 0.142244 \end{bmatrix}$ \
\
weights vector: $\underline{x} = \begin{bmatrix} 0.09579578 \\ 0.90420422 \end{bmatrix}$ \
\
weights vector transposed: $\underline{x}^T = \begin{bmatrix} 0.09579578 , 0.90420422 \end{bmatrix}$

Portfolio variance: $\sigma^2_P = \underline{x}^T \cdot M \cdot \underline{x}$

In [489]:
stdev = np.sqrt(variance)

print(f"Portfolio standard deviation is {stdev:.4f} or {(100 * stdev):.2f}%")

Portfolio standard deviation is 0.3546 or 35.46%


## Sharpe Ratio of Portfolio ##

In [490]:
# Let's say we bought 5 Bitcoin and 100 Ethereum five years ago.
start = datetime.date.today() - datetime.timedelta(5 * 365)
end = datetime.date.today()

prices = web.DataReader(["BTC-USD", "ETH-USD"], start, end)["Adj Close"]
prices

[*********************100%%**********************]  2 of 2 completed


,BTC-USD,ETH-USD
Date,,
2018-09-16,6517.180176,220.589005
2018-09-17,6281.200195,197.875000
2018-09-18,6371.299805,209.975006
2018-09-19,6398.540039,209.968994
2018-09-20,6519.669922,224.591003
...,...,...
2023-09-10,25832.226562,1616.828857
2023-09-11,25162.654297,1551.637695
2023-09-12,25833.343750,1592.429443


In [491]:
r_prices = prices.pct_change()
r_prices = r_prices.dropna()
r_prices

,BTC-USD,ETH-USD
Date,,
2018-09-17,-0.036209,-0.102970
2018-09-18,0.014344,0.061150
2018-09-19,0.004275,-0.000029
2018-09-20,0.018931,0.069639
2018-09-21,0.033020,0.097925
...,...,...
2023-09-10,-0.002450,-0.011212
2023-09-11,-0.025920,-0.040320
2023-09-12,0.026654,0.026289


In [492]:
initialBTC, finalBTC = (5*prices["BTC-USD"][0], 5*prices["BTC-USD"][-1])
initialETH, finalETH = (100*prices["ETH-USD"][0], 100*prices["ETH-USD"][-1])

BTCweight = initialBTC / (initialBTC + initialETH)
ETHweight = initialETH / (initialBTC + initialETH)

print(f"Weights of Bitcoin and Ethereum: ({BTCweight:.2f}, {ETHweight:.2f}) respectively.")

Weights of Bitcoin and Ethereum: (0.60, 0.40) respectively.


In [493]:
# Rates of return of the assets
rBTC = (finalBTC - initialBTC) / initialBTC
rETH = (finalETH - initialETH) / initialETH

print(f"Rate of return of BTC: {rBTC:.4f} or {(100*rBTC):.2f}% \
    \nRate of return of ETH: {rETH:.4f} or {(100*rETH):.2f}%")

Rate of return of BTC: 3.0723 or 307.23%     
Rate of return of ETH: 6.3756 or 637.56%


In [494]:
# Rate of return of the portfolio
rPortfolio = BTCweight * rBTC + ETHweight * rETH

print(f"Rate of return of Portfolio: {rPortfolio:.4f} or {(100*rPortfolio):.2f}%")

Rate of return of Portfolio: 4.4057 or 440.57%


In [495]:
# Calculating annual variance of the assets
varBTC = 365 * r_prices["BTC-USD"].var(ddof=1)
varETH = 365 * r_prices["ETH-USD"].var(ddof=1)

print(f"Annual variance of returns of BTC: {varBTC:.4f} or {(100*varBTC):.2f}% \
    \nAnnual variance of returns of ETH: {varETH:.4f} or {(100*varETH):.2f}%")

# Calculating annual variance of the portfolio
covariance_matrix = 365 * r_prices.cov()
print(covariance_matrix)

weights = np.array((BTCweight, ETHweight))

varPortfolio = np.dot(weights.T, np.dot(weights, covariance_matrix))
print(f"Variance of portfolio = {varPortfolio:.4f} or {100*(varPortfolio):.2f}%")

Annual variance of returns of BTC: 0.4652 or 46.52%     
Annual variance of returns of ETH: 0.7792 or 77.92%
          BTC-USD   ETH-USD
BTC-USD  0.465235  0.496638
ETH-USD  0.496638  0.779235
Variance of portfolio = 0.5315 or 53.15%


In [497]:
# Standard deviations
sigmaBTC = np.sqrt(varBTC)
sigmaETH = np.sqrt(varETH)
sigmaPortfolio = np.sqrt(varPortfolio)

# Average annual returns
avgannual_BTC = rBTC / 5
avgannual_ETH = rETH / 5
avgannual_Portfolio = rPortfolio / 5

# Sharpe ratios
sharpe_BTC = avgannual_BTC / sigmaBTC
sharpe_ETH = avgannual_ETH / sigmaETH
sharpe_Portfolio = avgannual_Portfolio / sigmaPortfolio

print(f"Sharpe ratio of BTC = {sharpe_BTC:.2f} \
    \nSharpe ratio of ETH = {sharpe_ETH:.2f} \
    \nSharpe ratio of Portfolio = {sharpe_Portfolio:.2f}")


Sharpe ratio of BTC = 0.90     
Sharpe ratio of ETH = 1.44     
Sharpe ratio of Portfolio = 1.21
